<a href="https://colab.research.google.com/github/Zewtta/PEL219_2025_RedesNeuraisArtificiais/blob/main/PEL219_MNIST_GLWT_CONVOLUCIONAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarefa 5
##Aluno: Matheus Vieira Martins
##Mestrado Eng. Mecânica

#### Anotações de aulas:
Reimplementem a rede de reconhecimento de dígitos
usando Convolução 2D:
- Pode ser uma simples.
- Pode ser a de 6 camadas, trocando 2 ou 3 por Conv2D.
- Pode ser a LeNet-5 (ver arquitetura nos slides).

Perguntas a responder:
- Quantos filtros por camada?
- Qual o tamanho do kernel de cada camada?

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import numpy as np

# ============================
# 1) Carregar e preparar dados
# ============================
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Adicionar canal (CNN exige 3D)
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)

# One-hot
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# ============================
# 2) Definir CNN
# ============================
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))  # Layer 0
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))                         # Layer 1
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

# ============================
# 3) Greedy Layer-Wise
# ============================
EPOCHS_PER_STAGE = 3
BATCH_SIZE = 32
early = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

for i in range(1, len(model.layers)):
    print(f"\n=== Treinando a partir da camada {i} ===")

    for layer in model.layers[:i]:
        layer.trainable = False
    for layer in model.layers[i:]:
        layer.trainable = True

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["categorical_accuracy"])

    model.fit(
        x_train, y_train,
        validation_split=0.1,
        epochs=EPOCHS_PER_STAGE,
        batch_size=BATCH_SIZE,
        callbacks=[early],
        verbose=2
    )

# ============================
# 4) Fine-Tuning final
# ============================
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["categorical_accuracy"])

model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=BATCH_SIZE,
    callbacks=[early],
    verbose=2
)

# ============================
# 5) Avaliação
# ============================
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nAccuracy final CNN + Greedy: {acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)


=== Treinando a partir da camada 1 ===
Epoch 1/3
